# Paths to our dataset files

In [1]:
data = '../datasets/bts_site_b_train/train.zip'
mapper = '../datasets/bts_site_b_train/mapper_TrainOnly.csv'
model = '../datasets/bts_site_b_train/Site_B.ttl'
schema = '../datasets/bts_site_b_train/Brick_v1.2.1.ttl'

# Load the Dataset

Hack for demo purposes to add the `src` directory to the path:

In [2]:
import sys
sys.path.append("../src")

Import and instantiate the manager:

In [3]:
from dbmgr import DBManager
db = DBManager(data, mapper, model, schema)

This `db` object is what is passed to each analytics module.

## Number of streams in the dataset

In [4]:
len(db)

668

## Iterating over the streams in the dataset

In [5]:
i = 0
for stream_id in db:
    print(stream_id)
    i += 1
    if i == 5:
        break

6f502ba0_77fb_43db_b6dc_48e68d5c822b
3c9ca09b_8952_45da_a066_bcb645cbbc68
d60c44c3_62fc_409a_a751_8b3ddeb9f197
eb0d5916_d942_4e2d_8129_60b83d984fdd
78078042_e16e_425b_9ec8_605aa149274e


## Check if a stream ID is in the dataset:

Not in the dataset:

In [6]:
'tim' in db

False

In the dataset:

In [7]:
'835a86d6_c235_4caf_8071_31b9bf24960b' in db

True

## Get a stream by its ID:

By indexing into the manager:

In [8]:
db['835a86d6_c235_4caf_8071_31b9bf24960b'].head()

,time,value,brick_class
0,2021-01-01 00:03:09.662,0.0,Enable_Status
1,2021-01-01 00:13:39.017,0.0,Enable_Status
2,2021-01-01 00:23:10.073,0.0,Enable_Status
3,2021-01-01 00:33:09.637,0.0,Enable_Status
4,2021-01-01 00:43:09.639,0.0,Enable_Status


By calling the getter:

In [9]:
db.get_stream('835a86d6_c235_4caf_8071_31b9bf24960b').head()

,time,value,brick_class
0,2021-01-01 00:03:09.662,0.0,Enable_Status
1,2021-01-01 00:13:39.017,0.0,Enable_Status
2,2021-01-01 00:23:10.073,0.0,Enable_Status
3,2021-01-01 00:33:09.637,0.0,Enable_Status
4,2021-01-01 00:43:09.639,0.0,Enable_Status


## Set a stream by its ID:

In [10]:
import pandas as pd

db['835a86d6_c235_4caf_8071_31b9bf24960b'] = pd.DataFrame({
            "time": [pd.Timestamp('2017-03-22 15:16:45'), pd.Timestamp('2017-03-22 15:17:45'), pd.Timestamp('2017-03-22 15:18:45'), pd.Timestamp('2017-03-22 15:19:45') ],
            "value": [22.0, 23.0, 24.0, 25.0],
            "brick_class": ["Fake_Class", "Fake_Class", "Fake_Class", "Fake_Class"],
        })

db['835a86d6_c235_4caf_8071_31b9bf24960b'].head()

,time,value,brick_class
0,2017-03-22 15:16:45,22.0,Fake_Class
1,2017-03-22 15:17:45,23.0,Fake_Class
2,2017-03-22 15:18:45,24.0,Fake_Class
3,2017-03-22 15:19:45,25.0,Fake_Class


## Get multiple streams by their IDs:

In [11]:
stream_ids = [
    '22543d51_3c38_46d3_98fe_a77838976df9',
    '3010a24a_2deb_43c8_9db4_cfc94c5faa5c',
    '405896a1_c195_422d_a0aa_18d2eca6a61d',
]
streams = db.get_streams(stream_ids)
for stream_id, stream in streams.items():
    print(f'========= {stream_id} =========')
    print(stream.head())
    print()

========= 22543d51_3c38_46d3_98fe_a77838976df9 =========
                     time  value            brick_class
0 2021-01-01 00:03:16.758    1.0  Operating_Mode_Status
1 2021-01-01 00:13:45.383    1.0  Operating_Mode_Status
2 2021-01-01 00:23:16.660    1.0  Operating_Mode_Status
3 2021-01-01 00:33:16.717    1.0  Operating_Mode_Status
4 2021-01-01 00:43:16.769    1.0  Operating_Mode_Status

========= 3010a24a_2deb_43c8_9db4_cfc94c5faa5c =========
                     time  value              brick_class
0 2021-07-13 01:26:18.083 -0.107  Electrical_Power_Sensor
1 2021-07-13 01:31:17.035 -0.042  Electrical_Power_Sensor
2 2021-07-13 01:36:13.143  0.054  Electrical_Power_Sensor
3 2021-07-13 01:50:12.735 -0.016  Electrical_Power_Sensor
4 2021-07-13 01:55:12.150 -0.164  Electrical_Power_Sensor

========= 405896a1_c195_422d_a0aa_18d2eca6a61d =========
                     time  value                   brick_class
0 2021-01-01 00:03:16.153   23.5  Cooling_Temperature_Setpoint
1 2021-01-01 00:1

## Query the building model:

Some arbitrary sparql query:

In [12]:
q = """
SELECT ?s ?p ?o WHERE {
    ?s ?p ?o
} LIMIT 5
"""

Get the results as RDFLib rows:

In [13]:
res = db.query(q)
for r in res:
    print(r)

(rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#da5873de_e112_4f9b_9e0f_149bd1221894.ef1a1240_07d1_4572_b3e5_6591c31372db'), rdflib.term.URIRef('http://senaps.io/schema/1.0/senaps#stream_id'), rdflib.term.Literal('9f6e47c8_ec19_402e_b27b_4551655ec984'))
(rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#c10f79c9_744d_40a5_a272_7cde7ca1f5a6.4b524f12_0c6f_4abd_a677_51db619f635f'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://brickschema.org/schema/Brick#Average_Zone_Air_Temperature_Sensor'))
(rdflib.term.BNode('n3e552ab4ff564ed192ed88e16624a27ab298'), rdflib.term.URIRef('https://brickschema.org/schema/Brick#value'), rdflib.term.Literal('ABC'))
(rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44

Get the results as a DataFrame:

In [14]:
res = db.query(q, return_df=True)
res.head()

,o,p,s
0,9f6e47c8_ec19_402e_b27b_4551655ec984,http://senaps.io/schema/1.0/senaps#stream_id,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...
1,https://brickschema.org/schema/Brick#Average_Z...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...
2,ABC,https://brickschema.org/schema/Brick#value,n3e552ab4ff564ed192ed88e16624a27ab298
3,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,https://brickschema.org/schema/Brick#isPointOf,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...
4,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,https://brickschema.org/schema/Brick#isPartOf,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...


## Query the brick schema:

Get the results as RDFLib rows:

In [15]:
res = db.query(q, graph='schema')
for r in res:
    print(r)

(rdflib.term.BNode('nce7879a9aa914fa8b69946c72bf6e1c5b2234'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#rest'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#nil'))
(rdflib.term.BNode('nce7879a9aa914fa8b69946c72bf6e1c5b5698'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#intersectionOf'), rdflib.term.BNode('nce7879a9aa914fa8b69946c72bf6e1c5b5699'))
(rdflib.term.URIRef('https://brickschema.org/schema/BrickTag#Evaporative'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://brickschema.org/schema/Brick#Tag'))
(rdflib.term.BNode('nce7879a9aa914fa8b69946c72bf6e1c5b7591'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#rest'), rdflib.term.BNode('nce7879a9aa914fa8b69946c72bf6e1c5b7592'))
(rdflib.term.BNode('nce7879a9aa914fa8b69946c72bf6e1c5b5991'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#intersectionOf'), rdflib.term.BNode('nce7879a9aa914fa8b69946c72bf6e1c5b5992'))


Get the results as a DataFrame:

In [16]:
res = db.query(q, graph='schema', return_df=True)
res.head()

,o,p,s
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#nil,http://www.w3.org/1999/02/22-rdf-syntax-ns#rest,nce7879a9aa914fa8b69946c72bf6e1c5b2234
1,nce7879a9aa914fa8b69946c72bf6e1c5b5699,http://www.w3.org/2002/07/owl#intersectionOf,nce7879a9aa914fa8b69946c72bf6e1c5b5698
2,https://brickschema.org/schema/Brick#Tag,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://brickschema.org/schema/BrickTag#Evapor...
3,nce7879a9aa914fa8b69946c72bf6e1c5b7592,http://www.w3.org/1999/02/22-rdf-syntax-ns#rest,nce7879a9aa914fa8b69946c72bf6e1c5b7591
4,nce7879a9aa914fa8b69946c72bf6e1c5b5992,http://www.w3.org/2002/07/owl#intersectionOf,nce7879a9aa914fa8b69946c72bf6e1c5b5991


## Remove the RDF prefixes from the query results:

In [17]:
res = db.query(q, return_df=True, defrag=True)
res.head()

,o,p,s
0,9f6e47c8_ec19_402e_b27b_4551655ec984,stream_id,da5873de_e112_4f9b_9e0f_149bd1221894.ef1a1240_...
1,Average_Zone_Air_Temperature_Sensor,type,c10f79c9_744d_40a5_a272_7cde7ca1f5a6.4b524f12_...
2,ABC,value,n3e552ab4ff564ed192ed88e16624a27ab298
3,8dbdff35_af3a_4c69_8963_53bdcb33c775,isPointOf,8dbdff35_af3a_4c69_8963_53bdcb33c775.5a8bbc3a_...
4,ebb4f301_f353_446c_9b92_19f131e5181d,isPartOf,76705912_de37_4fcb_b1bc_f32d9c6b032f


## Get the building model knowledge graph itself:

In [18]:
g = db.model
i = 0
for s, p, o in g:
    print(s, p, o)
    i += 1
    if i == 5:
        break

dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#da5873de_e112_4f9b_9e0f_149bd1221894.ef1a1240_07d1_4572_b3e5_6591c31372db http://senaps.io/schema/1.0/senaps#stream_id 9f6e47c8_ec19_402e_b27b_4551655ec984
dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#c10f79c9_744d_40a5_a272_7cde7ca1f5a6.4b524f12_0c6f_4abd_a677_51db619f635f http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://brickschema.org/schema/Brick#Average_Zone_Air_Temperature_Sensor
n3e552ab4ff564ed192ed88e16624a27ab298 https://brickschema.org/schema/Brick#value ABC
dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#8dbdff35_af3a_4c69_8963_53bdcb33c775.5a8bbc3a_d142_48cd_8c31_a76c56d4cc6f https://brickschema.org/schema/Brick#isPointOf dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8

## Get the brick schema knowledge graph itself:

In [19]:
g = db.schema
i = 0
for s, p, o in g:
    print(s, p, o)
    i += 1
    if i == 5:
        break

nce7879a9aa914fa8b69946c72bf6e1c5b2234 http://www.w3.org/1999/02/22-rdf-syntax-ns#rest http://www.w3.org/1999/02/22-rdf-syntax-ns#nil
nce7879a9aa914fa8b69946c72bf6e1c5b5698 http://www.w3.org/2002/07/owl#intersectionOf nce7879a9aa914fa8b69946c72bf6e1c5b5699
https://brickschema.org/schema/BrickTag#Evaporative http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://brickschema.org/schema/Brick#Tag
nce7879a9aa914fa8b69946c72bf6e1c5b7591 http://www.w3.org/1999/02/22-rdf-syntax-ns#rest nce7879a9aa914fa8b69946c72bf6e1c5b7592
nce7879a9aa914fa8b69946c72bf6e1c5b5991 http://www.w3.org/2002/07/owl#intersectionOf nce7879a9aa914fa8b69946c72bf6e1c5b5992


## Get the time series data itself:

In [20]:
data = db.data
i = 0
for k, v in data.items():
    print(f'{k}: {len(v)} data points')
    i += 1
    if i == 5:
        break

6f502ba0_77fb_43db_b6dc_48e68d5c822b: 51193 data points
3c9ca09b_8952_45da_a066_bcb645cbbc68: 51194 data points
d60c44c3_62fc_409a_a751_8b3ddeb9f197: 51194 data points
eb0d5916_d942_4e2d_8129_60b83d984fdd: 51200 data points
78078042_e16e_425b_9ec8_605aa149274e: 51188 data points
